<a href="https://colab.research.google.com/github/a-matson/DST_ASS1/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To-do



* Research, discuss and select performance metrics - ALL
* Introduce the dataset - ?
* EDA - Josh
* Everyone do their model experiments
  * Logistic regression - Geraint
  * SVM - Andre
  * Neural networks - Ethan
  * Decision trees - Josh
* Summary/conclusion - ?



# Imports

In [ ]:
!pip install

# Andre

## Logistic regression

## SMV

# Ethan

#Josh

#Geraint